# Slope Analysis

This project use the change of holding current slope to identify drug responders.

## Analysis Steps

The `getBaselineAndMaxDrugSlope` function smoothes the raw data by the moving window decided by `filterSize`, and analyzes the smoothed holding current in an ABF and returns baseline slope and drug slope.

The _slope of baseline_ is calculated as the linear regreasion slope during the 3 minutes period before drug onset.

In addition, the smoothed data are separated into segments which n = regressionSize data points are included. The linear regression slope is then calculated for each segment. 

The _peak slope of drug_ is the most negative slope during the chosen drug period (1-5 minutes after drug onset, in this case).

## Set-Up the Environment

In [ ]:
%load_ext autoreload
import sys
sys.path.append("../src")
from os.path import basename
import slopeTools
import plotTools
import statsTools
import matplotlib.pyplot as plt

## Define ABF Files and Filter Settings

The user can list the ABF files they want to analyze

In [ ]:
abfFilePaths = [
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804007.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804030.abf", 
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804043.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804048.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804060.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804066.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20805008.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20805029.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20805035.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20811011.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20811021.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20817012.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20831011.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20831017.abf"
]

The users can decide the parameters they want for data analysis. 

`filterSize` decides number of points (sweeps) for the moving window average. 

`regressionSize` decides the number of smoothed data points used to calculate linear regression slopes during the drug range.

In [ ]:
filterSize = 15
regressionSize = 15

## Analyze All ABFs

In [ ]:
baselineSlopes = []
drugSlopes = []
abfIDs = []
for abfFilePath in abfFilePaths:
    baselineSlope, drugSlope = slopeTools.getBaselineAndMaxDrugSlope(abfFilePath, filterSize, regressionSize)
    baselineSlopes.append(baselineSlope)
    drugSlopes.append(drugSlope)
    abfIDs.append(basename(abfFilePath))

## Compare Baseline vs. Drug Slopes

The users can plot the basleine slope and the peak drug slope of each cell, and report the p-value in the title by performing a paired t-test between baseline slopes and peak drug slopes.

In [ ]:
plotTools.plotPairs(baselineSlopes, drugSlopes, "slopes")

## Assess Responsiveness of All Cells

Generate a scatter plot showing the slope difference of each cell.

In [ ]:
slopeThreshold = -1.5 
drugEffects = []
for i in range(len(abfIDs)):
    drugEffects.append(drugSlopes[i] - baselineSlopes[i])

plt.figure (figsize=(6, 4))
plt.ylabel("Slope Difference (pA/min)")
plt.plot(abfIDs, drugEffects, 'o', color = "b")
plt.xticks(rotation = 45)
plt.axhline(slopeThreshold, color='r', ls='--')
plt.show()

## Define Cells as Responsive vs. Non-Responsive

The users can define the <b>slopeThreshold</b>. The difference between baseline slope and peak drug slope must be more negative than this value to be a responder
slopeThreshold

In [ ]:
drugEffects=statsTools.responderByDelta(abfIDs, drugEffects, slopeThreshold)